In [13]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [14]:
from data_objects.article import Article
from data_objects.legislation import Legislation,ArticleMapping
from data_objects.public_consultation import PublicConsultation
from data_objects.base import Base

import requests
from sqlalchemy import create_engine, select, union_all, func, update, and_
from sqlalchemy.orm import Session, aliased

from bs4 import BeautifulSoup
import time

In [15]:
from configparser import ConfigParser

config = ConfigParser()
config.read("../config.ini")

['../config.ini']

In [16]:
engine = create_engine(config.get('QUERY_REFERENCE', 'db_file'))

ORM Query Samples

Simple where clause

In [17]:
with Session(engine) as sess:
    stmt = select(Legislation).where(Legislation.id <10)
    legislations = sess.scalars(stmt)
    l = legislations.all()
print(l)


[Legislation(id=1, title=Ολοκλήρωση της δημόσιας ηλεκτρονικής διαβούλευσης για το σχέδιο νόμου του Υπουργείου Εσωτερικών με τίτλο «ΑΡΣΗ ΠΕΡΙΟΡΙΣΜΩΝ ΓΙΑ ΤΗΝ ΕΓΓΡΑΦΗ ΣΤΟΥΣ ΕΙΔΙΚΟΥΣ ΕΚΛΟΓΙΚΟΥΣ ΚΑΤΑΛΟΓΟΥΣ ΕΚΛΟΓΕΩΝ ΕΞΩΤΕΡΙΚΟΥ»., ministry=Υπουργείο Εσωτερικών, date_posted=2023-07-10), Legislation(id=2, title=Ανοιχτή δημόσια διαβούλευση του Οδηγού Προσβασιμότητας για ιστοτόπους και εφαρμογές για φορητές συσκευές των οργανισμών της Ελληνικής Δημόσιας Διοίκησης, ministry=Υπουργείο Ψηφιακής Διακυβέρνησης, date_posted=2023-04-10), Legislation(id=3, title=Ολοκλήρωση διαβούλευσης για το σχέδιο νόμου με τίτλο «Ρυθμίσεις σχετικά με τους Οργανισμούς Τοπικής Αυτοδιοίκησης α’ και β’ βαθμού – Ρυθμίσεις σχετικά με τον ειδικό εκλογικό χώρο για ΑμεΑ και το δικαίωμα του εκλέγεσθαι – Διατάξεις για την ευζωία των ζώων συντροφιάς – Διατάξεις για το ανθρώπινο δυναμικό του δημοσίου τομέα – Λοιπές ρυθμίσεις του Υπουργείου Εσωτερικών», ministry=Υπουργείο Εσωτερικών, date_posted=2023-03-30), Legislation(id=4, title=

Join Statement

In [18]:
with Session(engine) as sess:
    # Join Statement
    stmt = select(Legislation, Article).where(Legislation.id == 1).join(Article.legislation)
    rows = sess.execute(stmt)
    for row in rows:
        print(f"{row.Legislation.id} {row.Article.number} {row.Article.title}")

1 1 Άρθρο 1 Άρση περιορισμών για την εγγραφή στους ειδικούς εκλογικούς καταλόγους εκλογέων εξωτερικού– Τροποποίηση άρθρων 2 και 4 ν. 4648/2019
1 2 Άρθρο 2 Έναρξη ισχύος


selecting invidivual attributes

In [19]:
with Session(engine) as sess:
    stmt = select(Legislation.id, Article.title).join(Article.legislation).where(Legislation.id == 2)
    rows = sess.execute(stmt)
    for row in rows:
        print(f"{row.id}, {row.title}")

2, Οδηγός Προσβασιμότητας για ιστοτόπους και εφαρμογές για φορητές συσκευές των οργανισμών της Ελληνικής Δημόσιας Διοίκησης


Subqueries

In [20]:
with Session(engine) as sess:
    inner_stmt = select(Legislation).where(Legislation.id<10)
    subq = inner_stmt.subquery()
    aliased_legislation = aliased(Legislation, subq)
    stmt = select(aliased_legislation)
    rows = sess.execute(stmt)
    for leg_obj in rows.scalars():
        print(leg_obj)


Legislation(id=1, title=Ολοκλήρωση της δημόσιας ηλεκτρονικής διαβούλευσης για το σχέδιο νόμου του Υπουργείου Εσωτερικών με τίτλο «ΑΡΣΗ ΠΕΡΙΟΡΙΣΜΩΝ ΓΙΑ ΤΗΝ ΕΓΓΡΑΦΗ ΣΤΟΥΣ ΕΙΔΙΚΟΥΣ ΕΚΛΟΓΙΚΟΥΣ ΚΑΤΑΛΟΓΟΥΣ ΕΚΛΟΓΕΩΝ ΕΞΩΤΕΡΙΚΟΥ»., ministry=Υπουργείο Εσωτερικών, date_posted=2023-07-10)
Legislation(id=2, title=Ανοιχτή δημόσια διαβούλευση του Οδηγού Προσβασιμότητας για ιστοτόπους και εφαρμογές για φορητές συσκευές των οργανισμών της Ελληνικής Δημόσιας Διοίκησης, ministry=Υπουργείο Ψηφιακής Διακυβέρνησης, date_posted=2023-04-10)
Legislation(id=3, title=Ολοκλήρωση διαβούλευσης για το σχέδιο νόμου με τίτλο «Ρυθμίσεις σχετικά με τους Οργανισμούς Τοπικής Αυτοδιοίκησης α’ και β’ βαθμού – Ρυθμίσεις σχετικά με τον ειδικό εκλογικό χώρο για ΑμεΑ και το δικαίωμα του εκλέγεσθαι – Διατάξεις για την ευζωία των ζώων συντροφιάς – Διατάξεις για το ανθρώπινο δυναμικό του δημοσίου τομέα – Λοιπές ρυθμίσεις του Υπουργείου Εσωτερικών», ministry=Υπουργείο Εσωτερικών, date_posted=2023-03-30)
Legislation(id=4, title=Δημό

Union operator

In [21]:
with Session(engine) as sess:
    u = union_all(
        select(Legislation).where(Legislation.id <5),
        select(Legislation).where(Legislation.id == 5)
    )
    stmt = select(Legislation).from_statement(u)
    for leg_obj in sess.execute(stmt).scalars():
        print(leg_obj)

Legislation(id=1, title=Ολοκλήρωση της δημόσιας ηλεκτρονικής διαβούλευσης για το σχέδιο νόμου του Υπουργείου Εσωτερικών με τίτλο «ΑΡΣΗ ΠΕΡΙΟΡΙΣΜΩΝ ΓΙΑ ΤΗΝ ΕΓΓΡΑΦΗ ΣΤΟΥΣ ΕΙΔΙΚΟΥΣ ΕΚΛΟΓΙΚΟΥΣ ΚΑΤΑΛΟΓΟΥΣ ΕΚΛΟΓΕΩΝ ΕΞΩΤΕΡΙΚΟΥ»., ministry=Υπουργείο Εσωτερικών, date_posted=2023-07-10)
Legislation(id=2, title=Ανοιχτή δημόσια διαβούλευση του Οδηγού Προσβασιμότητας για ιστοτόπους και εφαρμογές για φορητές συσκευές των οργανισμών της Ελληνικής Δημόσιας Διοίκησης, ministry=Υπουργείο Ψηφιακής Διακυβέρνησης, date_posted=2023-04-10)
Legislation(id=3, title=Ολοκλήρωση διαβούλευσης για το σχέδιο νόμου με τίτλο «Ρυθμίσεις σχετικά με τους Οργανισμούς Τοπικής Αυτοδιοίκησης α’ και β’ βαθμού – Ρυθμίσεις σχετικά με τον ειδικό εκλογικό χώρο για ΑμεΑ και το δικαίωμα του εκλέγεσθαι – Διατάξεις για την ευζωία των ζώων συντροφιάς – Διατάξεις για το ανθρώπινο δυναμικό του δημοσίου τομέα – Λοιπές ρυθμίσεις του Υπουργείου Εσωτερικών», ministry=Υπουργείο Εσωτερικών, date_posted=2023-03-30)
Legislation(id=4, title=Δημό

Specify Join

In [22]:
with Session(engine) as sess:
    stmt = select(Legislation).join(Article, Legislation.id == Article.legislation_id)
    print(stmt)

SELECT "Legislation".id, "Legislation".title, "Legislation".ministry, "Legislation".date_posted, "Legislation".parliament_url, "Legislation".legislation_pdf_url, "Legislation".scrap_url, "Legislation".final_legislation_id, "Legislation".gov_gazzete_number, "Legislation".legislation_type, "Legislation".law_number, "Legislation".no_final_legislation_reason, "Legislation".is_public_consultation 
FROM "Legislation" JOIN "Article" ON "Legislation".id = "Article".legislation_id


Specify Join Multiple Criteria

In [36]:
stmt = select(Legislation).where(Legislation.is_public_consultation==1) \
                          .join(ArticleMapping, and_(Article.legislation_id==ArticleMapping.legislation_id,
                                                     Article.number>=ArticleMapping.public_consultation_article_no))
print(stmt)

SELECT "Legislation".id, "Legislation".title, "Legislation".ministry, "Legislation".date_posted, "Legislation".parliament_url, "Legislation".legislation_pdf_url, "Legislation".scrap_url, "Legislation".final_legislation_id, "Legislation".gov_gazzete_number, "Legislation".legislation_type, "Legislation".law_number, "Legislation".no_final_legislation_reason, "Legislation".is_public_consultation 
FROM "Legislation" JOIN "ArticleMapping" ON "Article".legislation_id = "ArticleMapping".legislation_id AND "Article".number >= "ArticleMapping".public_consultation_article_no 
WHERE "Legislation".is_public_consultation = :is_public_consultation_1


Custom ON criteria

In [24]:
with Session(engine) as sess:
    stmt = select(Legislation).join(Article.legislation.and_(Article.comments_allowed==True))
    print(stmt)

SELECT "Legislation".id, "Legislation".title, "Legislation".ministry, "Legislation".date_posted, "Legislation".parliament_url, "Legislation".legislation_pdf_url, "Legislation".scrap_url, "Legislation".final_legislation_id, "Legislation".gov_gazzete_number, "Legislation".legislation_type, "Legislation".law_number, "Legislation".no_final_legislation_reason, "Legislation".is_public_consultation 
FROM "Article" JOIN "Legislation" ON "Legislation".id = "Article".legislation_id AND "Article".comments_allowed = true


Join to Subquery

In [25]:
# https://docs.sqlalchemy.org/en/20/orm/queryguide/select.html#joining-to-subqueries
with Session(engine) as sess:
    subq = select(Article).where(Article.comments_allowed==False).subquery()
    stmt = select(Legislation).join(subq, subq.c.legislation_id == Legislation.id)
    print(stmt)
    # In order to return subquery results, we need to alias the subq
    articles_with_comments = aliased(Article,subq, name="articles")
    stmt = select(Legislation,articles_with_comments).join(articles_with_comments).where(Legislation.id==3)
    for row in sess.execute(stmt):
        print(f"{row.articles}")

SELECT "Legislation".id, "Legislation".title, "Legislation".ministry, "Legislation".date_posted, "Legislation".parliament_url, "Legislation".legislation_pdf_url, "Legislation".scrap_url, "Legislation".final_legislation_id, "Legislation".gov_gazzete_number, "Legislation".legislation_type, "Legislation".law_number, "Legislation".no_final_legislation_reason, "Legislation".is_public_consultation 
FROM "Legislation" JOIN (SELECT "Article".id AS id, "Article".number AS number, "Article".title AS title, "Article".text AS text, "Article".legislation_id AS legislation_id, "Article".comments_allowed AS comments_allowed, "Article".article_url AS article_url 
FROM "Article" 
WHERE "Article".comments_allowed = false) AS anon_1 ON anon_1.legislation_id = "Legislation".id
Article(id=5, number=2, title=Άρθρο 2 Αντικείμενο, text= Αντικείμενο του παρόντος Μέρους είναι:α) η εισαγω, legislation_id=3)
Article(id=7, number=4, title=Άρθρο 4 Διεξαγωγή υπαίθριων αθλητικών εκδηλώσεων και διοργανώσεων, text= 1. 

Any (for one-to-many relationships)

In [26]:
with Session(engine) as sess:
    stmt = select(Legislation).where(Legislation.articles.any(Article.number==9999))
    print(stmt)
    # Negation
    stmt = select(Legislation).where(~Legislation.articles.any(Article.comments_allowed==0))
    print(stmt)

SELECT "Legislation".id, "Legislation".title, "Legislation".ministry, "Legislation".date_posted, "Legislation".parliament_url, "Legislation".legislation_pdf_url, "Legislation".scrap_url, "Legislation".final_legislation_id, "Legislation".gov_gazzete_number, "Legislation".legislation_type, "Legislation".law_number, "Legislation".no_final_legislation_reason, "Legislation".is_public_consultation 
FROM "Legislation" 
WHERE EXISTS (SELECT 1 
FROM "Article" 
WHERE "Legislation".id = "Article".legislation_id AND "Article".number = :number_1)
SELECT "Legislation".id, "Legislation".title, "Legislation".ministry, "Legislation".date_posted, "Legislation".parliament_url, "Legislation".legislation_pdf_url, "Legislation".scrap_url, "Legislation".final_legislation_id, "Legislation".gov_gazzete_number, "Legislation".legislation_type, "Legislation".law_number, "Legislation".no_final_legislation_reason, "Legislation".is_public_consultation 
FROM "Legislation" 
WHERE NOT (EXISTS (SELECT 1 
FROM "Article" 

Has (for many-to-one relationships)

In [27]:
with Session(engine) as sess:
    stmt = select(PublicConsultation).where(PublicConsultation.articles.has(Article.number==999))
    print(stmt)

SELECT "PublicConsultation".id, "PublicConsultation".reporter, "PublicConsultation".text, "PublicConsultation".date_reported, "PublicConsultation".url, "PublicConsultation".article_id 
FROM "PublicConsultation" 
WHERE EXISTS (SELECT 1 
FROM "Article" 
WHERE "Article".id = "PublicConsultation".article_id AND "Article".number = :number_1)


Order By

In [28]:
with Session(engine) as session:
    stmt = select(Article).order_by(Article.id.desc()) # Article.id.asc() for ascending
    print(stmt)

SELECT "Article".id, "Article".number, "Article".title, "Article".text, "Article".legislation_id, "Article".comments_allowed, "Article".article_url 
FROM "Article" ORDER BY "Article".id DESC


Group by / Having


In [29]:
with Session(engine) as sess:
    stmt = select(Legislation.scrap_url, func.count(Legislation.id), func.min(Legislation.id))\
            .group_by(Legislation.scrap_url).having(func.count(Legislation.id) > 1)
    print(stmt)
    rows = sess.execute(stmt)
    for row in rows:
        print(f"{row}")


SELECT "Legislation".scrap_url, count("Legislation".id) AS count_1, min("Legislation".id) AS min_1 
FROM "Legislation" GROUP BY "Legislation".scrap_url 
HAVING count("Legislation".id) > :count_2
('https://www.hellenicparliament.gr/Nomothetiko-Ergo/Anazitisi-Nomothetikou-Ergou?law_id=185752e7-a8d4-4866-8e58-ab64016dbe0a', 2, 958)
('https://www.hellenicparliament.gr/Nomothetiko-Ergo/Anazitisi-Nomothetikou-Ergou?law_id=21ea0129-0b28-4479-b6d5-a839013de48c', 2, 1050)
('https://www.hellenicparliament.gr/Nomothetiko-Ergo/Anazitisi-Nomothetikou-Ergou?law_id=2c5c4057-1da6-4b5d-953c-a49100ef17ca', 2, 1137)
('https://www.hellenicparliament.gr/Nomothetiko-Ergo/Anazitisi-Nomothetikou-Ergou?law_id=40be718c-1d8b-4ecb-a705-ab17016ca330', 2, 966)
('https://www.hellenicparliament.gr/Nomothetiko-Ergo/Anazitisi-Nomothetikou-Ergou?law_id=410420c1-af4d-4141-9d1c-bc30c496a968', 2, 1177)
('https://www.hellenicparliament.gr/Nomothetiko-Ergo/Anazitisi-Nomothetikou-Ergou?law_id=4790edbe-1692-44a1-af79-ae7100e90

Add Column Alias

In [30]:
with Session(engine) as sess:
    stmt = select(Legislation.scrap_url, func.count(Legislation.id).label("myLabel"), func.min(Legislation.id))\
            .group_by(Legislation.scrap_url).having(func.count(Legislation.id) > 1)
    print(stmt)
    rows = sess.execute(stmt)
    for row in rows:
        print(f"{row}")

SELECT "Legislation".scrap_url, count("Legislation".id) AS "myLabel", min("Legislation".id) AS min_1 
FROM "Legislation" GROUP BY "Legislation".scrap_url 
HAVING count("Legislation".id) > :count_1
('https://www.hellenicparliament.gr/Nomothetiko-Ergo/Anazitisi-Nomothetikou-Ergou?law_id=185752e7-a8d4-4866-8e58-ab64016dbe0a', 2, 958)
('https://www.hellenicparliament.gr/Nomothetiko-Ergo/Anazitisi-Nomothetikou-Ergou?law_id=21ea0129-0b28-4479-b6d5-a839013de48c', 2, 1050)
('https://www.hellenicparliament.gr/Nomothetiko-Ergo/Anazitisi-Nomothetikou-Ergou?law_id=2c5c4057-1da6-4b5d-953c-a49100ef17ca', 2, 1137)
('https://www.hellenicparliament.gr/Nomothetiko-Ergo/Anazitisi-Nomothetikou-Ergou?law_id=40be718c-1d8b-4ecb-a705-ab17016ca330', 2, 966)
('https://www.hellenicparliament.gr/Nomothetiko-Ergo/Anazitisi-Nomothetikou-Ergou?law_id=410420c1-af4d-4141-9d1c-bc30c496a968', 2, 1177)
('https://www.hellenicparliament.gr/Nomothetiko-Ergo/Anazitisi-Nomothetikou-Ergou?law_id=4790edbe-1692-44a1-af79-ae7100e

using functions (e.g. select count(*) from table)

In [31]:
with Session(engine) as sess:
    stmt = select(func.count()).select_from(Legislation)
    result = sess.execute(stmt).all()
    print(result)
    print(len(result))

[(1268,)]
1


In [32]:
stmt = select(func.lower(Legislation.title))
print(stmt)

# func is as liberal as possible to support any function    
stmt = select(func.some_crazy_func(Legislation.ministry,True,17))
print(stmt)

SELECT lower("Legislation".title) AS lower_1 
FROM "Legislation"
SELECT some_crazy_func("Legislation".ministry, :some_crazy_func_2, :some_crazy_func_3) AS some_crazy_func_1 
FROM "Legislation"


Update

In [33]:
old_title="Ολοκλήρωση της δημόσιας ηλεκτρονικής διαβούλευσης για το σχέδιο νόμου του Υπουργείου Εσωτερικών με τίτλο «ΑΡΣΗ ΠΕΡΙΟΡΙΣΜΩΝ ΓΙΑ ΤΗΝ ΕΓΓΡΑΦΗ ΣΤΟΥΣ ΕΙΔΙΚΟΥΣ ΕΚΛΟΓΙΚΟΥΣ ΚΑΤΑΛΟΓΟΥΣ ΕΚΛΟΓΕΩΝ ΕΞΩΤΕΡΙΚΟΥ»."
with Session(engine) as sess:
    sess.execute(
        update(Legislation),
        [
            {"id":1, "title":"SomeRandomTitle"}  # Dictionary must include primary key
        ]
    )
    stmt = select(Legislation).where(Legislation.id==1)
    rows = sess.execute(stmt).scalars()
    for row in rows:
        print(f"{row}")  # Verify that the title has indeed changed
    
    print()

    sess.execute(
        update(Legislation),
        [
            {"id":1, "title":old_title}
        ]
    )
    stmt = select(Legislation).where(Legislation.id==1)
    rows = sess.execute(stmt).scalars()
    for row in rows:
        print(f"{row}")  # Verify that the title has been restored to the old one
    sess.commit()


Legislation(id=1, title=SomeRandomTitle, ministry=Υπουργείο Εσωτερικών, date_posted=2023-07-10)

Legislation(id=1, title=Ολοκλήρωση της δημόσιας ηλεκτρονικής διαβούλευσης για το σχέδιο νόμου του Υπουργείου Εσωτερικών με τίτλο «ΑΡΣΗ ΠΕΡΙΟΡΙΣΜΩΝ ΓΙΑ ΤΗΝ ΕΓΓΡΑΦΗ ΣΤΟΥΣ ΕΙΔΙΚΟΥΣ ΕΚΛΟΓΙΚΟΥΣ ΚΑΤΑΛΟΓΟΥΣ ΕΚΛΟΓΕΩΝ ΕΞΩΤΕΡΙΚΟΥ»., ministry=Υπουργείο Εσωτερικών, date_posted=2023-07-10)


is None, in where clauses

In [34]:
# in clause
with Session(engine) as sess:
    stmt = select(Legislation).where(Legislation.id.in_([1,2,3]))
    legObjs = sess.execute(stmt).scalars().all()
    print(legObjs)

    # is None clause

    stmt = select(Legislation).where(Legislation.ministry.is_(None))
    legObjs = sess.execute(stmt).scalars().all()
    print(legObjs)



[Legislation(id=1, title=Ολοκλήρωση της δημόσιας ηλεκτρονικής διαβούλευσης για το σχέδιο νόμου του Υπουργείου Εσωτερικών με τίτλο «ΑΡΣΗ ΠΕΡΙΟΡΙΣΜΩΝ ΓΙΑ ΤΗΝ ΕΓΓΡΑΦΗ ΣΤΟΥΣ ΕΙΔΙΚΟΥΣ ΕΚΛΟΓΙΚΟΥΣ ΚΑΤΑΛΟΓΟΥΣ ΕΚΛΟΓΕΩΝ ΕΞΩΤΕΡΙΚΟΥ»., ministry=Υπουργείο Εσωτερικών, date_posted=2023-07-10), Legislation(id=2, title=Ανοιχτή δημόσια διαβούλευση του Οδηγού Προσβασιμότητας για ιστοτόπους και εφαρμογές για φορητές συσκευές των οργανισμών της Ελληνικής Δημόσιας Διοίκησης, ministry=Υπουργείο Ψηφιακής Διακυβέρνησης, date_posted=2023-04-10), Legislation(id=3, title=Ολοκλήρωση διαβούλευσης για το σχέδιο νόμου με τίτλο «Ρυθμίσεις σχετικά με τους Οργανισμούς Τοπικής Αυτοδιοίκησης α’ και β’ βαθμού – Ρυθμίσεις σχετικά με τον ειδικό εκλογικό χώρο για ΑμεΑ και το δικαίωμα του εκλέγεσθαι – Διατάξεις για την ευζωία των ζώων συντροφιάς – Διατάξεις για το ανθρώπινο δυναμικό του δημοσίου τομέα – Λοιπές ρυθμίσεις του Υπουργείου Εσωτερικών», ministry=Υπουργείο Εσωτερικών, date_posted=2023-03-30)]
[Legislation(id=1151, ti

Select top 1

In [35]:
with Session(engine) as sess:
    url = "https://www.hellenicparliament.gr/Nomothetiko-Ergo/Anazitisi-Nomothetikou-Ergou?law_id=fd683678-97a0-4d4e-9b6f-7e2658321207"
    stmt = select(Legislation).where(Legislation.scrap_url==url)

    r = sess.execute(stmt).scalars().first()
    print(r)

Legislation(id=1201, title=Ρυθμίσεις θεμάτων Ανανεώσιμων Πηγών Ενέργειας και άλλες διατάξεις., ministry=Περιβάλλοντος, Ενέργειας και Κλιματικής Αλλαγής, date_posted=2013-10-24)
